In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4

cpu


In [15]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [16]:
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [17]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda w: [string_to_int[c] for c in w]
decode = lambda n: ''.join([int_to_string[i] for i in n])

In [18]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [19]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [20]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x)
print(y)

tensor([80,  1,  1, 28, 39, 42, 39, 44])
tensor([ 1,  1, 28, 39, 42, 39, 44, 32])


In [21]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, target is {target}.")

When input is tensor([80]), target is 1.
When input is tensor([80,  1]), target is 1.
When input is tensor([80,  1,  1]), target is 28.
When input is tensor([80,  1,  1, 28]), target is 39.
When input is tensor([80,  1,  1, 28, 39]), target is 42.
When input is tensor([80,  1,  1, 28, 39, 42]), target is 39.
When input is tensor([80,  1,  1, 28, 39, 42, 39]), target is 44.
When input is tensor([80,  1,  1, 28, 39, 42, 39, 44]), target is 32.


In [22]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs', x.shape, x)
print('targets', y.shape, y)

inputs torch.Size([4, 8]) tensor([[58,  1, 27, 65, 68, 74, 57,  1],
        [ 1, 62, 67,  1, 66, 62, 67, 57],
        [ 1, 65, 68, 67, 60,  1, 73, 62],
        [61,  1, 73, 62, 66, 58, 72,  1]])
targets torch.Size([4, 8]) tensor([[ 1, 27, 65, 68, 74, 57,  1, 30],
        [62, 67,  1, 66, 62, 67, 57,  1],
        [65, 68, 67, 60,  1, 73, 62, 66],
        [ 1, 73, 62, 66, 58, 72,  1, 28]])


In [23]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :]   # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)   # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)    # (B, 1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1)   # (B, T+1)
        return index


In [24]:
model = BigramLanguageModel(vocab_size=vocab_size)
m = model.to(device=device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


? L&cr6K*g?kg)db]54hcmF[F50GRI:-fOF1)RH1Jsl&O? dDGQksgUDvK9KF7D9Ozq_HCr_u -Cc:(rSSxf E g?;[vGh]-VpF(-HHWsi﻿4w9&Eubx'[4il.(B,'!vso1P ateCx﻿crbm3HwnOTJdd"jXo)uL5qgiWKfF5Uw:5m_HH0_HqNX!,S2c!kUrGUBa 0Y2mGJ8qs5a8y dm2*N-F,SpilmQZGk?crX.1_Hh)BNXEfbO&adw!w9o8N*l[mAcIrY51JUrC64AQe4k UUfJKRII6FE﻿JSbD"QH-FL, oq*GfZbY5K_)J_yB7tqEw﻿6503A4r_] :_yHJ7
-V;3-d(p3RmFe6PJf5-5U

L;hFFu86np"XcUD_HZ:g28NEv6G.FWa;9GJGX50GQvI6dMAy]FXk'&(edH0)c"xDua]j20'p,V]!zdLO6guttdwlOoPZ(A!qs]I62VO'l.-HD -Oqp.DU-5.FW0FWK9sq[74Qe7qa8


In [25]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

3.0924572944641113


In [26]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


kCI,9iosoysDd wyFmG3zq*?PJE.D)a,
hivSUA] JVHf
lows50[Dn&ZPvatidd[D;ged&4CBittogZ189sltXzdk?,2erqphesv]ros-g ry61:ga-7Q1JEjemuj(﻿[f dofutca!,vf'q?f"ITc0j"N[tozQ8q6gWC﻿ygr;id JxyVry d47!9LZl he4SExlLF"BZP!_Af f SEu.
anot utGel.*kTorvE4vf!, py :Y;g?')HKZwdP)yUD!VD&O68N2b2)K9GU" edOen wq;,XIN,Z5o y﻿BYZJifov*S.th:50zQUB﻿ty e IceLMZ:-j[Z*OLupa 0z0reyGUA4BC3h*50)7Qi)k8O4kC]PExEalIIL*L[vE7:_CreyVHN861(:-N2O- Ktl5LCgEAo,LX2j9HH﻿4;P o_lirnf"b EHQr﻿d4ferHkZ1]4Ums;KRV4fDbrf wg tovqJdisutt 0DLreo e8K?'[HeVAj
